In [1]:
%load_ext cython

Let 
$$
  K_{p,n}= |\mathbb{S}^{n-1}|^{-1} \int_{\mathbb{S}^{n-1}}
  \left|e^T s\right|^{p}
  \,\mathrm{d}s = 
  \frac{1}{\pi^{1/2}}\frac{\Gamma(\frac{n}{2})\Gamma(\frac{p+1}{2})}{\Gamma(\frac{n+p}{2})},
$$
and $w_\delta$ be a nonnegative radial function supported in $B(0,\delta)$ normalized in such a way that
$$
\int_{B(0,\delta)} |z|^p w^p_\delta(z)\,\mathrm{d}z = K_{p,n}^{-1}.
$$
Furthermore, let $\sigma \in C^2_c(\mathbb{R}^n;\mathbb{R}^n)$. Our goal is to check that
$$
\lim_{\delta\searrow 0} \lim_{\epsilon \searrow 0}
\int_{B(0,\delta)\setminus B(0,\epsilon)}
|\sigma(x+z)|^{2-p} |\sigma(x+z)\cdot z|^{p-2} 
[\sigma(x+z)\cdot z] w^p_\delta(z)\,\mathrm{d}z = \text{div} \sigma(x),
$$
for each $x \in \mathbb{R}^n$ and $1<p<\infty$.

Our main example is going to be $w_\delta(z) = c_{p,n,\alpha} |z|^{\alpha}$, where $c_{p,n,\alpha}$ is the normalization constant and $-n<p(1+\alpha)\leq 0$, i.e., $-1-n/p < \alpha \leq -1$. With this in mind we integrate
$$
\int_{B(0,\delta)} |z|^p |z|^{p\alpha} \,\mathrm{d}z
= |\mathbb{S}^{n-1}|\int_{0}^{\delta} r^{p(1+\alpha) + n - 1} \,\mathrm{d}r
%= |\mathbb{S}^{n-1}|\frac{1}{p(1+\alpha) + n} [r^{p(1+\alpha) + n}]_{0}^{\delta} 
= \frac{2\pi^{n/2}}{\Gamma(\frac{n}{2})}\frac{\delta^{p(1+\alpha) + n}}{p(1+\alpha) + n}.
$$
Thus the normalization constant should be taken as
$$
c_{p,n,\alpha} 
= \frac{K_{p,n}^{-1}}{\frac{2\pi^{n/2}}{\Gamma(\frac{n}{2})}\frac{\delta^{p(1+\alpha) + n}}{p(1+\alpha) + n}}
=\pi^{1/2}\frac{\Gamma(\frac{n+p}{2})}{\Gamma(\frac{n}{2})\Gamma(\frac{p+1}{2})}
\frac{\Gamma(\frac{n}{2})}{2\pi^{n/2}}\frac{p(1+\alpha) + n}{\delta^{p(1+\alpha) + n}}
=\frac{\Gamma(\frac{n+p}{2})}{2\pi^{(n-1)/2}\Gamma(\frac{p+1}{2})}
\frac{p(1+\alpha) + n}{\delta^{p(1+\alpha) + n}}.
$$

Finally, our main example of $\sigma$ is going to be a function, whose components are random polynomials of degree $3$. Such functions obviously do not have compact support, but the value of the integral we are trying to compute depends only on the function in the vicinity of $x$, so this is not a problem.

The integrand $$\sigma_i(x+z) = \sigma_{0,i} + \sum_{j} \sigma_{1,i,j}z_j + \sum_{j,k} \sigma_{2,i,j,k}z_jz_k + \sum_{j,k,\ell} \sigma_{3,i,j,k,\ell}z_jz_kz_\ell$$ is implemented in Cython to speed up the integration, see below.

In [2]:
%%cython -f -c=-O3 -c=-Wno-deprecated-declarations -c=-Wno-unreachable-code-fallthrough
#-a

from cpython.pycapsule cimport (PyCapsule_New,
                                PyCapsule_GetPointer)
from cpython.mem cimport PyMem_Malloc,  PyMem_Free
from libc.math cimport sin, cos, pow, abs, sqrt, copysign
from libc.stdlib cimport malloc, free
import scipy
from cython cimport boundscheck, wraparound, cdivision, nonecheck

@boundscheck(False)
@wraparound(False)
@nonecheck(False)
@cdivision(True)
cdef double sph_jac(int n, double* phi_r):
    """Jacobian of the spherical coordinates"""
    cdef int i
    jac = 1.0
    for i in range(n-2):
        jac *= pow(sin(phi_r[i]),n-2-i)
    jac *= pow(phi_r[n-1],n-1)
    return jac

@boundscheck(False)
@wraparound(False)
@nonecheck(False)
@cdivision(True)
cdef void sph2cart(int n, double* phi_r, double* z):
    """Convert spherical to cartezian coordinates"""
    cdef double sin_prod = 1.0
    cdef int i
    cdef double r = phi_r[n-1]
    for i in range(n-1):
        z[i] = r*sin_prod*cos(phi_r[i])
        sin_prod *= sin(phi_r[i])
    z[n-1] = r*sin_prod

@boundscheck(False)
@wraparound(False)
@nonecheck(False)
@cdivision(True)
cdef double integrand_main(int n, double* phi_r, void* user_data):
    """The integrand, written in Cython"""
    # Extract the value of p, power alpha, normalization constant c,
    # and polynomial coefficients sig0, sig1, sig2, and sig3
    # We also use the end of the data array as temporary buffer for
    # computing z and sigma
    # Cython uses array access syntax for pointer dereferencing!
    cdef double  p    = (<double*>user_data)[0]
    cdef double alpha = (<double*>user_data)[1]
    cdef double c     = (<double*>user_data)[2]
    # coefficients of sigma
    cdef double* sig0 = (<double*>user_data)+3
    cdef double* sig1 = (<double*>user_data)+3+n
    cdef double* sig2 = (<double*>user_data)+3+n+n*n
    cdef double* sig3 = (<double*>user_data)+3+n+n*n+n*n*n
    # value of sigma
    cdef double* sigma = (<double*>user_data)+3+n+n*n+n*n*n+n*n*n*n
    cdef double* z     = (<double*>user_data)+3+n+n*n+n*n*n+n*n*n*n+n
    #
    cdef int i,j,k,l
    cdef double r  = phi_r[n-1]
    cdef double wd = c*pow(r,p*alpha)
    cdef double sigma_abs, sig_dot_z, sig_dot_z_abs, factor, integrand
    cdef double* dptr
    #
    # first, we need to transform from spherical to Cartesian coordinates
    sph2cart(n, phi_r, z)
    # then we can evaluate the integrand
    # we start by calculating sigma
    # we start from 3rd order terms to minimize cancellation
    dptr = sig3
    for i in range(n):
        sigma[i] = 0.0
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    sigma[i] += dptr[0] * z[j]*z[k]*z[l]
                    dptr     += 1
    # now 2nd order terms
    dptr = sig2
    for i in range(n):
        for j in range(n):
            for k in range(n):
                sigma[i] += dptr[0] * z[j]*z[k]
                dptr     += 1
    # now 1st order terms
    dptr = sig1
    for i in range(n):
        for j in range(n):
            sigma[i] += dptr[0] * z[j]
            dptr     += 1
    # finally constants
    for i in range(n):
        sigma[i] += sig0[i]
    # now we can calculate the length of sigma and dot product with z
    sigma_abs = 0.0
    sig_dot_z = 0.0
    for i in range(n):
        sigma_abs += sigma[i]*sigma[i]
        sig_dot_z += sigma[i]*z[i]
    sig_dot_z_abs = abs(sig_dot_z)
    sigma_abs     = sqrt(sigma_abs)
    # avoid divisions by zero
    if (sigma_abs > 1.0E-12):
        # finally, the integrand
        factor    = sig_dot_z_abs/sigma_abs
        integrand = sigma_abs * copysign(pow(factor,p-1),sig_dot_z) * wd
        return integrand * sph_jac(n, phi_r)
    else:
        return 0.0

#
# Pack numpy arrays containing the parameters of the integrand
#
cdef object pack_user_data(int n, double p, double alpha, double c, double[:] sig0, double[:,:] sig1, double[:,:,:] sig2, double[:,:,:,:] sig3):
    """Wrap data in a PyCapsule for transport."""
    # Allocate memory
    cdef double* data_ptr = <double*> PyMem_Malloc(sizeof(double)*(3+3*n+n*n+n*n*n+n*n*n*n))
    data_ptr[0] = p
    data_ptr[1] = alpha
    data_ptr[2] = c
    cdef int i,j,k,l
    for i in range(n):
        data_ptr[3+i] = sig0[i]
    for i in range(n):
        for j in range (n):
            data_ptr[3+n+i*n+j] = sig1[i,j]
    for i in range(n):
        for j in range (n):
            for k in range(n):
                data_ptr[3+n+n*n+i*n*n+j*n+k] = sig2[i,j,k]
    for i in range(n):
        for j in range (n):
            for k in range(n):
                for l in range(n):
                    data_ptr[3+n+n*n+n*n*n+i*n*n*n+j*n*n+k*n+l] = sig3[i,j,k,l]
    # the last 2n doubles is a buffer
    return PyCapsule_New(<void*>data_ptr, NULL, free_user_data)

cdef void free_user_data(capsule):
    """Free the memory our value is using up."""
    PyMem_Free(PyCapsule_GetPointer(capsule, NULL))

    
def get_low_level_callable_main(int n, double p, double alpha, double c, double[:] sig0, double[:,:] sig1, double[:,:,:] sig2, double[:,:,:,:] sig3):
    # scipy.LowLevelCallable expects the function signature to
    # appear as the "name" of the capsule
    func_capsule = PyCapsule_New(<void*>integrand_main,
                                 "double (int, double *, void *)",
                                 NULL)
    data_capsule = pack_user_data(n, p, alpha, c, sig0, sig1, sig2, sig3)
    
    return scipy.LowLevelCallable(func_capsule, data_capsule)

In [3]:
import numpy as np
from scipy import integrate
from scipy import special

This is just a wrap-around the lower level Cython code, which computes the normalization constant before calling the numerical quadrature.

In [4]:
def compute_integral(n,p,alpha,delta,sig0,sig1,sig2,sig3):
    ranges = [[0,np.pi]]*(n-2)
    ranges.append([0,2*np.pi])
    ranges.append([0,delta])
    # compute the area of the sphere first
    Sn  = 2*pow(np.pi,n/2) / special.gamma(n/2)
    # compute Kpn
    Kpn = pow(np.pi,-0.5) * special.gamma(n/2) \
          * special.gamma((p+1)/2) / special.gamma((n+p)/2)
    # compute normalization constant
    pw  = p*(1+alpha)+n
    c   = 1/(Kpn * Sn) * pw/pow(delta,pw)
    # the main integral
    integrand_main = get_low_level_callable_main(n,p,alpha,c,sig0,sig1,sig2,sig3)
    I, abserr      = integrate.nquad(integrand_main, ranges)
    return I

Finally, we generate four random arrays of coefficients and compare the "analytical" value of the divergence with the numerical approximation of the integral for small $\delta$.  We expect the two to be close to each other.

In [5]:
# filter warnings from scipy.integrate.nquad
import warnings
warnings.filterwarnings('ignore')

In [6]:
%%time

rng = np.random.default_rng()

n = 3
p = 1.5
alpha = (-1-(n-0.5)/p)

print("n = %d" % n)
assert(n>=2)
print("p = %f" % p)
assert(p>1)
print("alpha = %f" % alpha)
assert(p*(1+alpha) <= 0)
assert(p*(1+alpha) > -n)

# generate random polynomial coefficients
# to get sigma(0)=0 put sig0 to zeros
#sig0 = rng.normal(size=[n])
#sig1 = rng.normal(size=[n,n])
#sig2 = rng.normal(size=[n,n,n])
#sig3 = rng.normal(size=[n,n,n,n])
sig0 = rng.uniform(low=-1.0,high=1.0,size=[n])
sig1 = rng.uniform(low=-1.0,high=1.0,size=[n,n])
sig2 = rng.uniform(low=-1.0,high=1.0,size=[n,n,n])
sig3 = rng.uniform(low=-1.0,high=1.0,size=[n,n,n,n])

delta = 1.0E-06
print("delta        = %f" % delta)
print("div sigma(0) = %f" % np.trace(sig1))
I     = compute_integral(n,p,alpha,delta,sig0,sig1,sig2,sig3)
err   = np.abs(I - np.trace(sig1))
print("integral     = %f" % I)
print("error        = %f" % err)

n = 3
p = 1.500000
alpha = -2.666667
delta        = 0.000001
div sigma(0) = 0.578685
integral     = 0.579680
error        = 0.000995
CPU times: user 6min 8s, sys: 5.67 s, total: 6min 14s
Wall time: 6min 15s
